In [ ]:
#https://github.com/whitew1994WW/LangGraphForBeginners/blob/main/tutorial_react.ipynb

In [10]:
from typing import Annotated, Literal
from typing_extensions import TypedDict
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode
from langchain_openai import ChatOpenAI


In [11]:
# pwd

In [12]:
# !pip install -r requirements.txt #all at once

In [19]:
pip install python-dotenv==1.0.1

  Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
Note: you may need to restart the kernel to use updated packages.


In [20]:

import os
from dotenv import load_dotenv
import logging
# Load environment variables
load_dotenv()


# Get API keys and other environment variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [21]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

graph = StateGraph(State)



In [22]:


@tool
def get_weather(location: str):
    """Call to get the current weather."""
    if location.lower() in ["yorkshire"]:
        return "It's cold and wet."
    else:
        return "It's warm and sunny."
    
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    model="gpt-4o",
)

tools = [get_weather]

llm_with_tools = llm.bind_tools(tools)

tool_node = ToolNode(tools)

graph.add_node("tool_node", tool_node)



In [23]:


def prompt_node(state: State) -> State:
    new_message = llm_with_tools.invoke(state["messages"])
    return {"messages": [new_message]}

graph.add_node("prompt_node", prompt_node)



In [24]:
def conditional_edge(state: State) -> Literal['tool_node', '__end__']:
    last_message = state["messages"][-1]
    if last_message.tool_calls:
        return "tool_node"
    else:
        return "__end__"
    
graph.add_conditional_edges(
    'prompt_node',
    conditional_edge
)
graph.add_edge("tool_node", "prompt_node")
graph.set_entry_point("prompt_node")

In [ ]:
APP = graph.compile()

new_state = APP.invoke({"messages": ["Whats the weather in yorkshire?"]})

print(new_state["messages"][-1].content)

The weather in Yorkshire is currently cold and wet.
